#1


In [1]:
!pip install razdel

In [31]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
import requests

In [32]:
news = requests.get('https://raw.githubusercontent.com/anastasie57/compling_57/refs/heads/main/compling/lenta.txt').text

In [33]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


In [34]:
from collections import Counter

In [35]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [36]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [37]:
sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [41]:
sentences_news

[['<start>',
  'бои',
  'у',
  'сопоцкина',
  'и',
  'друскеник',
  'закончились',
  'отступлением',
  'германцев',
  '<end>'],
 ['<start>',
  'неприятель',
  'приблизившись',
  'с',
  'севера',
  'к',
  'осовцу',
  'начал',
  'артиллерийскую',
  'борьбу',
  'с',
  'крепостью',
  '<end>'],
 ['<start>',
  'в',
  'артиллерийском',
  'бою',
  'принимают',
  'участие',
  'тяжелые',
  'калибры',
  '<end>'],
 ['<start>',
  'с',
  'раннего',
  'утра',
  '14',
  'сентября',
  'огонь',
  'достиг',
  'значительного',
  'напряжения',
  '<end>'],
 ['<start>',
  'попытка',
  'германской',
  'пехоты',
  'пробиться',
  'ближе',
  'к',
  'крепости',
  'отражена',
  '<end>'],
 ['<start>', 'в', 'галиции', 'мы', 'заняли', 'дембицу', '<end>'],
 ['<start>',
  'большая',
  'колонна',
  'отступавшая',
  'по',
  'шоссе',
  'от',
  'перемышля',
  'к',
  'саноку',
  'обстреливалась',
  'с',
  'высот',
  'нашей',
  'батареей',
  'и',
  'бежала',
  'бросив',
  'парки',
  'обоз',
  'и',
  'автомобили',
  '<end>'],

In [51]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, 3))

In [23]:
from scipy.sparse import lil_matrix

In [72]:
matrix_news = lil_matrix((len(bigrams_news),
                        len(bigrams_news)))

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}


for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    matrix_news[bigram2id_news[word1 + ' ' + word2], word2id_news[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[word1 + ' ' + word2])

In [76]:
def generate(matrix, id2bigram, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]

    for i in range(n):

        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        #chosen = matrix[current_idx].argmax()
        text.append(id2bigram[chosen])

        if '<end>' in id2bigram[chosen]:
            chosen = word2id['<start>']
        current_idx = chosen

    return ' '.join(text)

In [79]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [99]:
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)

    return prob, len(tokens)


def compute_join_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)

    return prob, len(tokens)

In [97]:
generated_texts = [generate(matrix_news, id2bigram_news, word2id_news).replace(' <end>', '.').replace(' <start>', '.') for _ in range(5)]

In [100]:
generated_texts_str = '. '.join(generated_texts)

In [101]:
ps = []
for sent in sent_tokenize(generated_texts_str):
    prob, N = compute_join_proba_markov_assumption(sent, unigrams_news, bigrams_news)
    if not N:
        continue
    ps.append(perplexity(prob, N))

In [102]:
np.mean(ps)

13355.747394243037

#2

In [104]:
from typing import List

In [105]:
class Beam:
    sequence: List[str]
    score: float
    def __init__(self, sequence: List[str], score: float) -> None:
        self.sequence: list = sequence
        self.score: float = score
        return None

In [117]:
def generate_with_beam_search(matrix, id2bigram, bigram2id, start, n=100, max_beams=5):
    initial_node = Beam(sequence=[start], score=np.log1p(0))
    beams = [initial_node]

    for i in range(n):
        new_beams = []
        for beam in beams:
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue

            last_id = bigram2id[beam.sequence[-1]]
            probas = matrix[last_id].toarray()[0]

            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            for top_id in top_idxs:
                if not probas[top_id]:
                    break

                new_sequence = beam.sequence + [id2bigram[top_id]]
                new_score = beam.score + np.log1p(probas[top_id])
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]

    best_sequence = max(beams, key=lambda x: x.score).sequence

    return ' '.join(best_sequence)

In [118]:
print(generate_with_beam_search(matrix_news, id2bigram_news, bigram2id_news, start='россия не'))

россия не столько отказов бывший комендант <start> победа направив могучий <start> австрийцы этом погиб года штабс-капитан соответственным в продолжая преследовать и прочая при продолжающемся бежала бросив перемышля к <start> большая продолжающемся отступлении бросив парки к саноку большая колонна в галиции м ю <start> неприятель приблизившись с севера к осовцу начал артиллерийскую борьбу с севера <start> неприятель приблизившись с севера к осовцу начал артиллерийскую борьбу с севера <start> неприятель приблизившись с севера к осовцу начал артиллерийскую борьбу с севера <start> неприятель приблизившись с севера к осовцу начал артиллерийскую борьбу с севера <start> неприятель приблизившись с севера к осовцу начал артиллерийскую борьбу с севера <start> неприятель приблизившись с севера к осовцу начал артиллерийскую борьбу с севера <start> неприятель приблизившись с севера к осовцу начал артиллерийскую борьбу с севера <start> неприятель приблизившись с севера к осовцу начал артиллерийскую

отличия от beam search по одному слову: весь текст все так же крутится вокруг одной темы, но чаще встречаются осмысленные связки из пары слов